# Example 1:

Below is an example of how to use the package with the two defined model architectures, convolution, and FF.

## Imports

In [1]:
from mm_neural_adjoint import NANetwork, ConvModel, LinModel
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import numpy as np

## Read example data

In [3]:
X = pd.read_csv('data/data_x_tiny.csv', header=None, delimiter=' ')
y = pd.read_csv('data/data_y_tiny.csv', header=None, delimiter=' ')

X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32)

## Convert to Dataloaders

In [4]:
dataset = TensorDataset(X_tensor, y_tensor)

# Split the dataset into train, validation, and test
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size, test_size]
)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

## Train

This code cell will train the forward pass of the model.

Model is saved during training.

Geometries -> Spectra

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ConvModel(8, 300)

model = NANetwork(model, device=device)

epochs = 50

with tqdm(total=epochs, desc='Training Progress') as pbar:
    model.train(epochs, train_loader, val_loader, progress_bar=pbar, save=True)


# model.evaluate_geometry(test_loader)

Training Progress:   0%|          | 0/50 [00:00<?, ?it/s]2025/07/01 16:09:24 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/07/01 16:09:24 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Training Progress: 100%|██████████| 50/50 [00:06<00:00,  7.78it/s, train_loss=0.015785, val_loss=0.021309]


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = LinModel(8, 300)

model = NANetwork(model, device=device)

epochs = 50

with tqdm(total=epochs, desc='Training Progress') as pbar:
    model.train(epochs, train_loader, val_loader, progress_bar=pbar, save=True)


# model.evaluate_geometry(test_loader)

Training Progress: 100%|██████████| 50/50 [00:07<00:00,  6.83it/s, train_loss=0.009565, val_loss=0.020639]


## Load Model

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LinModel(8, 300)
model = NANetwork(model, device=device)


model.load('checkpoints/best_model.pt')
print(model.geometry_lower_bound)

Successfully loaded model from checkpoints/best_model.pt
tensor([-1., -1., -1., -1., -1., -1., -1., -1.])


## Normal Prediction

Geometry -> Spectra

In [8]:
spectra = model.predict_spectra(X_tensor[0].unsqueeze(0))
print(spectra.shape)

(1, 300)


## NA Method

Spectra -> Geometry

In [9]:
Xpred_top, Ypred_top, MSE_top = model.predict_geometry(y_tensor[0])
print(Xpred_top.shape, Ypred_top.shape, MSE_top.shape)


(1, 8) (1, 300) (1,)


# Example 2

### Define any pytorch model to use with NANetwork

In [2]:
class SimpleModel(nn.Module):
    def __init__(self, input_size=200, output_size=10, hidden_size=128):
        super(SimpleModel, self).__init__()
        
        # Define the layers
        self.layers = nn.Sequential(
            # Input layer: 200 -> 128
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            # Hidden layer: 128 -> 64
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            # Hidden layer: 64 -> 32
            nn.Linear(hidden_size // 2, hidden_size // 4),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            # Output layer: 32 -> 10
            nn.Linear(hidden_size // 4, output_size)
        )
    
    def forward(self, x):
        return self.layers(x)

### Create some simple synthetic training data

In [7]:
# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Generate synthetic training data
n_samples = 1000
n_features = 200
n_outputs = 10

# Create input features (X) - random values between -1 and 1
X = torch.randn(n_samples, n_features) * 0.5  # Normal distribution with std=0.5

# Create a simple relationship for outputs (Y)
# This creates a non-linear relationship between inputs and outputs
def generate_outputs(X):
    # Create some non-linear transformations
    Y = torch.zeros(n_samples, n_outputs)
    
    for i in range(n_outputs):
        # Each output depends on different combinations of input features
        feature_indices = torch.randperm(n_features)[:20]  # Use 20 random features per output
        weights = torch.randn(20) * 0.1
        
        # Non-linear transformation
        Y[:, i] = torch.sum(X[:, feature_indices] * weights, dim=1) + \
                  torch.sin(torch.sum(X[:, feature_indices], dim=1)) * 0.1 + \
                  torch.randn(n_samples) * 0.05  # Add some noise
    
    return Y

# Generate outputs
y = generate_outputs(X)

# Convert to pandas DataFrames (optional, for easy viewing)
X_df = pd.DataFrame(X.numpy())
y_df = pd.DataFrame(y.numpy())

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"X range: [{X.min():.3f}, {X.max():.3f}]")
print(f"y range: [{y.min():.3f}, {y.max():.3f}]")

# Create DataLoader
dataset = TensorDataset(X, y)

# Split into train/validation/test
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.29 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size, test_size]
)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

print(f"\nDataset splits:")
print(f"Train: {len(train_dataset)} samples")
print(f"Validation: {len(val_dataset)} samples")
print(f"Test: {len(test_dataset)} samples")

X shape: torch.Size([1000, 200])
y shape: torch.Size([1000, 10])
X range: [-2.295, 2.315]
y range: [-1.052, 0.850]

Dataset splits:
Train: 700 samples
Validation: 290 samples
Test: 10 samples


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sample_X, sample_y = next(iter(train_loader))

model_base = SimpleModel(sample_X.shape[1], sample_y.shape[1])

model = NANetwork(model_base, device=device)

epochs = 50

with tqdm(total=epochs, desc='Training Progress') as pbar:
    model.train(epochs, train_loader, val_loader, progress_bar=pbar, save=True)


# model.evaluate_geometry(test_loader)

Training Progress: 100%|██████████| 50/50 [00:01<00:00, 35.75it/s, train_loss=0.016397, val_loss=0.017917]


### Results will be saved in val_results

In [10]:
model.evaluate_geometry(test_loader)

The model is standard NA model, falling back to first layer extraction


Evaluating geometries: 100%|██████████| 10/10 [00:15<00:00,  1.59s/it]


('val_results/val_Ypred.csv', 'val_results/val_Ytruth.csv')

In [11]:
Xpred_top, Ypred_top, MSE_top = model.predict_geometry(y_tensor[0])
print(Xpred_top.shape, Ypred_top.shape, MSE_top.shape)

NameError: name 'y_tensor' is not defined